### HMM三个基本问题

----------
#### 1.概率计算问题
盒子和球模型λ=(A,B,π)，状态集合Q=｛1，2，3｝，观测集合V=｛红，白｝，

A = [ 0.5 0.2 0.3           

     0.3 0.5 0.2
     
     0.2 0.3 0.5]
     
B = [0.5 0.5

     0.4 0.6
     
     0.7 0.3]
     
π=(0.2,0.4,0.4)^T

设T=3，0=（红，白，红）。计算P(O|π)

In [40]:
import numpy as np
####### 1.1 前向算法 ########
# 0.数值化
A = np.array([[0.5,0.2,0.3],
              [0.3,0.5,0.2],
              [0.2,0.3,0.5]])
B = np.array([[0.5,0.5],
              [0.4,0.6],
              [0.7,0.3]])
pi = np.array([0.2,0.4,0.4]).reshape((3,1))
lambd = [A,B,pi]
Q = [0,1,2]
V = ['r','w']
V = [0,1]

T = 3
O = ('r','w','r')
O = [0,1,0]

def forwardCal(lambd,O):
    T = len(O) #时间序列长度
    Q = len(lambd[0][0,:])#隐含状态种类 
    #V = len(lambd[1][0,:])#观测状态种类
    alpha = np.zeros((T,Q))
    # 1.计算初值
    for i in range(T):
        alpha[O[0]][i] = pi[i]*lambd[1][i][O[0]]
    # 2.递推计算
    for i in range(1,T):
        for j in range(Q):
            alpha[i][j] = np.sum(alpha[i-1,:]*lambd[0][:,j])*lambd[1][j,O[i]]
    # 3.终止
    print(alpha)
    P = np.sum(alpha[-1,:])
    return P

forwardCal(lambd,O)

[[ 0.1       0.16      0.28    ]
 [ 0.077     0.1104    0.0606  ]
 [ 0.04187   0.035512  0.052836]]


0.130218

In [41]:
####### 1.2 后向算法 ########
def backwardCal(lambd,O):
    T = len(O) #时间序列长度
    Q = len(lambd[0][0,:])#隐含状态种类 
    #V = len(lambd[1][0,:])#观测状态种类
    beta = np.zeros((T,Q))
    # 1.赋初值
    beta[0,:] = 1.0  
    # 2.递推计算
    for i in range(1,T):
        for j in range(Q):
            beta[i][j] = np.sum(beta[i-1,:]*lambd[0][j,:].T*lambd[1][:,O[-i]])
    # 3.终止
    print(beta)
    P = np.sum(beta[-1,:]*pi.T*lambd[1][:,O[0]])
    return P

backwardCal(lambd,O)

[[ 1.      1.      1.    ]
 [ 0.54    0.49    0.57  ]
 [ 0.2451  0.2622  0.2277]]


0.130218

#### 2.学习问题

In [3]:
####### 2.1 Baum-Welch算法 ########
#同EM

#### 3.预测问题

In [42]:
####### 3.1 维特比算法 ########
#已知A,B,pi,O,求最优路径I*
def viterbi(lambd,O):
    A = lambd[0]
    B = lambd[1]
    pi = lambd[2]
    T = len(O) #时间序列长度
    Q = len(lambd[0][0,:])#隐含状态种类 
    
    delte = np.zeros((T,Q))
    psi = [0]
    
    #初始化：
    delte[0,:] = (pi*B[:,O[0]].reshape((3,1))).T
    
    #递推
    for t in range(1,T):
        delte[t,:] = np.max(delte[t-1,:]*A.T,axis=1)*B[:,O[t]]
        psi_tmp = np.argmax(delte[t-1,:]*A.T,axis=1)
        psi.append(psi_tmp)
    #终止
    print(delte)
    print(psi)
    P_end = np.max(delte[-1,:])
    i_end = np.argmax(delte[-1,:])
    I = []
    for p in psi[1:]:
        I.append(np.max(p))
    I.append(i_end)
    #回溯
    return I
    
    
viterbi(lambd,O)

[[ 0.1      0.16     0.28   ]
 [ 0.028    0.0504   0.042  ]
 [ 0.00756  0.01008  0.0147 ]]
[0, array([2, 2, 2], dtype=int64), array([1, 1, 2], dtype=int64)]


[2, 2, 2]